# Superposition Exploration

It's hypothesized that neural networks represent concepts as nearly-orthogonal vectors in a high-dimensional space, and this lets the network represent many more concepts than there are dimensions in the model's hidden space. This phenomenon is known as the "[superposition](https://transformer-circuits.pub/2022/toy_model/index.html)". The [Johnson-Lindenstrauss theorem](https://en.wikipedia.org/wiki/Johnson%E2%80%93Lindenstrauss_lemma) proves that as we increase the number of dimensions in a space, the number of almost-orthogonal vectors we can fit in the space increases exponentially.

But concretely, how many concepts can we fit in a space the size of a typical transformer model's hidden space? And how superposition interference should we expect for a given number of concepts in a given number of dimensions? In this notebook we'll explore these questions.

<img src="./assets/superposition.png" alt="Superposition diagram" width="300">

*From [Toy Models of Superposition](https://transformer-circuits.pub/2022/toy_model/index.html)*



## Defining superposition

We need a metric we can use to track how much superposition interference there is in a given space. We'll use **mean max absolute cosine similarity**, $\rho_\text{mm}$, as our core measure. For each vector in our space, we calculate the absolute value of cosine similarity between that vector and every other vector in the space, then take the mean across all these max values.

$$
\rho_\text{mm} = \frac{1}{N} \sum_{i=1}^{N} \max_{j \neq i} \left| \frac{\mathbf{v}_i \cdot \mathbf{v}_j}{\|\mathbf{v}_i\| \|\mathbf{v}_j\|} \right|
$$

This metric represents a "worst-case" measure of superposition interference for each vector in our space.

## Superposition of random vectors

If we have $N$ random unit-norm vectors in a $d$-dimensional space, what should we expect $\rho_\text{mm}$ to be? We can try this out with a simple simulation.

In [ ]:
import warnings
import torch
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tqdm.auto import tqdm

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    warnings.warn("No GPU or MPS device found, using CPU. This will be very slow!")


def compute_rho_mm(vectors: torch.Tensor, chunk_size: int = 4096) -> float:
    norms = vectors.norm(dim=1, keepdim=True)
    unit_vectors = vectors / norms
    N = unit_vectors.shape[0]

    # Process in chunks to avoid materializing the full (N, N) matrix
    max_abs_cos = torch.zeros(N, device=vectors.device)
    for i in range(0, N, chunk_size):
        chunk = unit_vectors[i : i + chunk_size]  # (chunk_size, d)
        cos_sim = chunk @ unit_vectors.T  # (chunk_size, N)

        # Zero out self-similarities on the diagonal block
        diag_start = i
        diag_end = min(i + chunk_size, N)
        cos_sim[:, diag_start:diag_end].fill_diagonal_(0.0)

        max_abs_cos[i : i + chunk_size] = cos_sim.abs().max(dim=1).values

    return max_abs_cos.mean().item()


dims = [256, 512, 768, 1024]
num_vectors = [4 * 1024, 8 * 1024, 16 * 1024, 32 * 1024]


results: dict[int, list[float]] = {N: [] for N in num_vectors}
for d in tqdm(dims, desc="Dimension"):
    for N in tqdm(num_vectors, desc=f"d={d}", leave=False):
        vectors = torch.randn(N, d, device=device)
        rho = compute_rho_mm(vectors)
        results[N].append(rho)

# Plot
fig, ax = plt.subplots(figsize=(8, 5))

norm = mcolors.LogNorm(vmin=min(num_vectors), vmax=max(num_vectors))
cmap = plt.cm.viridis

for N in num_vectors:
    color = cmap(norm(N))
    ax.plot(dims, results[N], marker="o", color=color, label=f"N = {N:,}")

ax.set_xlabel("Dimension (d)")
ax.set_ylabel(r"$\rho_\mathrm{mm}$")
ax.set_title(r"Mean Max Absolute Cosine Similarity ($\rho_\mathrm{mm}$) vs Dimension")

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label("Number of vectors (N)")
cbar.set_ticks(num_vectors)
cbar.set_ticklabels([f"{N:,}" for N in num_vectors])

ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

If we want to go even bigger, we'll run into trouble. Fortunately, we can compute the expected $\rho_\text{mm}$ exactly (up to numerical integration) using the known distribution of cosine similarity between random unit vectors.

For two random unit vectors in $\mathbb{R}^d$, the squared cosine similarity follows a Beta distribution: $\cos^2\theta \sim \text{Beta}(1/2, (d-1)/2)$. This means the CDF of $|\cos\theta|$ is the regularized incomplete beta function:

$$P(|\cos\theta| \leq t) = I_{t^2}\!\left(\tfrac{1}{2}, \tfrac{d-1}{2}\right)$$

For each vector, its max absolute cosine similarity with $N-1$ others has CDF $F(t)^{N-1}$ (treating pairwise similarities as independent, which is excellent for large $d$). The expected value of a non-negative random variable gives us:

$$\mathbb{E}[\rho_\text{mm}] = \int_0^1 \left(1 - \left[I_{t^2}\!\left(\tfrac{1}{2}, \tfrac{d-1}{2}\right)\right]^{N-1}\right) dt$$

Let's see how well this matches our simulation.

In [ ]:
from scipy import integrate
from scipy.special import betainc


def predicted_rho_mm(N: int, d: int) -> float:
    """Compute expected rho_mm using exact distribution of cosine similarity."""
    a, b = 0.5, (d - 1) / 2

    def integrand(t: float) -> float:
        cdf = betainc(a, b, t**2)
        return 1.0 - cdf ** (N - 1)

    result, _ = integrate.quad(integrand, 0.0, 1.0)
    return result


fig, ax = plt.subplots(figsize=(8, 5))

norm = mcolors.LogNorm(vmin=min(num_vectors), vmax=max(num_vectors))
cmap = plt.cm.viridis

dims_smooth = list(range(dims[0], dims[-1] + 1))

for N in num_vectors:
    color = cmap(norm(N))
    # Simulated
    ax.plot(dims, results[N], marker="o", color=color, label=f"N = {N:,} (simulated)")
    # Predicted (exact)
    predicted = [predicted_rho_mm(N, d) for d in dims_smooth]
    ax.plot(dims_smooth, predicted, color=color, linestyle="--", alpha=0.7, label=f"N = {N:,} (predicted)")

ax.set_xlabel("Dimension (d)")
ax.set_ylabel(r"$\rho_\mathrm{mm}$")
ax.set_title(r"Simulated vs Predicted $\rho_\mathrm{mm}$")

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label("Number of vectors (N)")
cbar.set_ticks(num_vectors)
cbar.set_ticklabels([f"{N:,}" for N in num_vectors])

ax.legend(fontsize=7, ncol=2)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

This is a pefect match! Let's use this to see how much superposition interference we should expect for a some really massive numbers of concepts, let's go up to 10 trillion concepts (10^13) and 8192 dimensions, which is the hidden size of the largest current open models. 10 trillion concepts seems like a reasonable upper bound for the max number of concepts that could conceivably be possible, since that would be roughly 1 concept per training token in a typical LLM pretraining run.

In [ ]:
import numpy as np
from tqdm.auto import tqdm

big_dims = [512, 768, 1024, 2048, 3072, 4096, 6144, 8192]
big_N = [10**k for k in range(5, 14, 2)]  # 10^5, 10^7, 10^9, 10^11, 10^13

big_results: dict[int, list[float]] = {}
for N in tqdm(big_N, desc="N values"):
    rho_values: list[float] = []
    for d in big_dims:
        rho_values.append(predicted_rho_mm(N, d))
    big_results[N] = rho_values

# Plot: d on x-axis, N as colored lines
fig, ax = plt.subplots(figsize=(8, 5))

norm = mcolors.LogNorm(vmin=min(big_N), vmax=max(big_N))
cmap = plt.cm.viridis

for N in big_N:
    color = cmap(norm(N))
    exp = int(round(np.log10(N)))
    ax.plot(big_dims, big_results[N], marker="o", markersize=4, color=color, label=f"N = $10^{{{exp}}}$")

ax.set_xlabel("Dimension (d)")
ax.set_ylabel(r"$\rho_\mathrm{mm}$")
ax.set_title(r"Superposition interference ($\rho_\mathrm{mm}$) vs hidden dimension")

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label("Number of vectors (N)")

ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

*10 trillion concepts* in 8192 dimensions has far less superposition interference than just *100K concepts* in 768 dimensions (the hidden dimension of GPT-2)! Also, at any given dimension, increasing the number of concepts by 100x doesn't even increase superposition interference much!

<h1 align="center" style="font-size: 100px;">🤯</h1>

## What if we optimally placed the vectors instead?

Everything above assumes random unit vectors. But what if we could arrange them optimally — placing each vector to minimize the worst-case interference? Would we do significantly better?

From [spherical coding theory](https://en.wikipedia.org/wiki/Spherical_code), the answer is: **barely**. The minimum achievable max pairwise correlation for $N$ optimally-placed unit vectors in $d$ dimensions is given by the [spherical cap packing bound](https://en.wikipedia.org/wiki/Spherical_cap#Solid_angle):

$$\varepsilon_\text{optimal} \approx \sqrt{1 - N^{-2/(d-1)}}$$

The intuition is that each vector "excludes" a spherical cap around itself, and we're counting how many non-overlapping caps fit on the unit sphere in $\mathbb{R}^d$.

When $\ln N \ll d$ (which holds for all practical settings — even $N = 10^{13}$ in $d = 512$ gives $\ln N / d \approx 0.06$), we can Taylor-expand:

$$N^{-2/(d-1)} = e^{-2\ln N/(d-1)} \approx 1 - \frac{2\ln N}{d}$$

which gives:

$$\varepsilon_\text{optimal} \approx \sqrt{\frac{2\ln N}{d}}$$

This is exactly the leading-order term of the random vector formula! So random placement is already near-optimal — there's essentially nothing to gain from clever geometric arrangement of the vectors, at least for the $N$ and $d$ values relevant to real neural networks.

This is a remarkable consequence of high-dimensional geometry: in spaces with hundreds or thousands of dimensions, random directions are already so close to orthogonal that you can't do meaningfully better by optimizing.